In [10]:
from multiprocessing.sharedctypes import Value
import networkx as nx
import numpy as np
import random
import pandas as pd

from tqdm import tqdm
from datetime import datetime
import pickle

# File imports
import data
from data import select_dataset, get_station_g, ingest_electricity_data, set_random_speed_columns
from replicate_graph import layer_graph
from vehicle import Vehicle
from simulation import Simulation
from visualization import set_draw_attributes

In [2]:
#open the pickle
with open('trials/wcctci_04_04_2022_12_15_28.pkl', 'rb') as inp:
    res = pickle.load(inp)

Get the highest and lowest utilized stations:

In [8]:
#get station utilization - which should be ordered in the order of the station id (0-42)
utilization = res.metrics['station_utilization']

#get the station that has the highest utilization.
h_st = np.argmax(utilization)

#get the station that has the lowest utilization.
l_st = np.argmin(utilization)

Find the unused location that is closest to the highly utilized station:

In [41]:
#get the map of all parking areas and stations
full_map_dist = pd.read_csv("data/charging_parking_distances.csv")

#keep only the roads that start or end with the high utilization station
h_map = full_map_dist[(full_map_dist.loc[:, 'OriginID'] == h_st) | (full_map_dist.loc[:, 'DestinationID'] == h_st)]

#then we eliminate all of the stations that are already being used. In this case it is the ids from 0 to 42 (except h_st)
#TO DO: think about the case in which we start with a configuration different from the wcctci so that the ones being used
#are not necesarily the 0-42. - would have to change the using_st bellow
using_st = list(range(0,43))
using_st.remove(h_st)
h_map = h_map[(~ h_map.loc[:, 'OriginID'].isin(using_st)) & (~ h_map.loc[:, 'DestinationID'].isin(using_st))]

#then we remove the case where we go from h_st to h_st
h_map = h_map[~ ((h_map.loc[:, 'OriginID'] == h_st) & (h_map.loc[:, 'DestinationID'] == h_st))]

#find the shortest edge. If there is more than one, we choose the first one that comes up (hence the 'reset_index' and [0] bellow)

new_origin = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['OriginID'][0]
new_destination = h_map[h_map.Total_TravelTime==h_map.Total_TravelTime.min()].reset_index()['DestinationID'][0]

#one of the two will be h_st and the other will be the new station location. 
if new_origin == h_st:
    new_station = new_destination
else:
    new_station = new_origin

Now we start to think about re-runing the simulation with this swap (taking out l_st and adding new_station). To do this, we need to create a new map with the locations and the distances of the stations that we will be using next.

In [ ]:
new_stations_list = using_st + [h_st, new_station]

In [ ]:
#we have "full_map_dist" (distances map)
#and we need the locations map
full_map_loc = pd.read_csv("data/charging_parking_stations.csv")

#now we reduce these to the new set of stations. All origins and destinations should be in this list.
new_map_dist = full_map_dist[(full_map_dist.loc[:, 'OriginID'].isin(new_stations_list)) & (full_map_dist.loc[:, 'DestinationID'].isin(new_stations_list))]
new_map_loc = full_map_loc[(full_map_loc.loc[:, 'OriginID'].isin(new_stations_list)) & (full_map_loc.loc[:, 'DestinationID'].isin(new_stations_list))]

Now we can run the next simulation

In [ ]:
#TODO : How would I do this?? I need to change something in the 'data.py'??

dataset = "wcctci"
simulation_length = 24
battery_interval = 20
km_per_percent = 6
sim = Simulation("wcctci", simulation_length, battery_interval, km_per_percent)